# NLU+ 2023-2024 Lab 3: Tensor Computation in PyTorch

#### Authors: Pasquale Minervini, Yao Fu, and Frank Keller

This lab indends to teach computation with tensors, which is a fundamental paradigm in modern machine learning. 

Students should work through Section 1 before the lab session, as this section introduces the basics of tensor computation. The lab session will then focus on more advanced computations with tensors, in Sections 2 and 3.

**Students are strongly encouraged to complete this lab before starting CW2. Many computations in the lab will be encountered in CW2 again, so the amount of difficult in the coursework will be significantly reduced.**

We suggest using jupyter lab instead of jupyter notebook. Basically jupyter lab is an enhanced version of jupyter notebook. Downloading jupyter lab is simply:

```bash
pip install jupyterlab
```

Then in a terminal, start jupyter lab with 
```bash
jupyter lab
```

In [1]:
import torch 
import torch.nn.functional as F

# Section 1. Basic Tensor Computation

## Definition of Tensor: High-dimensional Array

### Vector

A vector is a rank-1 (or one-dimentional) tensor.

In NLP, it could be a single sentence where each word is an integer index.

In [2]:
sent = torch.tensor([1, 3, 5, 7, 9])

In [3]:
sent

tensor([1, 3, 5, 7, 9])

### Matrix

A matrix is a rank-2 (or two-dimentional) tensor.

In NLP, this could used to represent a batch of different sentences.

In [4]:
sents = torch.tensor([[1, 3, 5, 7, 9], 
                     [2, 4, 6, 8, 10]]) # a batch of two sentences

In [5]:
sents # we assume these index corresponds to some actual words

tensor([[ 1,  3,  5,  7,  9],
        [ 2,  4,  6,  8, 10]])

### Rank-3 Tensor

There can be a rank-3 (three-dimensional) tensor.

In NLP, this could be a batch of sentences, where each word in each sentence is representened as an embedding vector.

In [6]:
sents_emb = torch.rand([2, 5, 10])  # [batch_size = 2, sentence_length = 5, hidden_size = 10]
                                    # we use random vectors for the purpose of demonstration 
                                    # we use the name `hidden_size` because the embedding is usually referred as 'hidden representations' in neural networks

In [7]:
sents_emb

tensor([[[0.0165, 0.4610, 0.2935, 0.7918, 0.9897, 0.5539, 0.9382, 0.9258,
          0.1409, 0.6015],
         [0.9948, 0.9281, 0.0693, 0.9243, 0.0479, 0.0031, 0.9342, 0.8005,
          0.0294, 0.0470],
         [0.6854, 0.2645, 0.7973, 0.1580, 0.6953, 0.1219, 0.7219, 0.6394,
          0.2284, 0.7700],
         [0.3644, 0.9829, 0.5922, 0.3716, 0.2549, 0.9234, 0.9027, 0.3655,
          0.9783, 0.2015],
         [0.4519, 0.4519, 0.1742, 0.3138, 0.3291, 0.8921, 0.3761, 0.8290,
          0.7847, 0.0578]],

        [[0.6409, 0.7583, 0.7778, 0.7661, 0.9191, 0.1258, 0.5029, 0.1685,
          0.9733, 0.4825],
         [0.4110, 0.0821, 0.2093, 0.5083, 0.7708, 0.1336, 0.6952, 0.9294,
          0.0439, 0.7541],
         [0.4254, 0.7400, 0.5253, 0.5484, 0.5981, 0.5491, 0.9301, 0.2295,
          0.1450, 0.3191],
         [0.9057, 0.6761, 0.8336, 0.6161, 0.6311, 0.6301, 0.2490, 0.5835,
          0.7767, 0.3806],
         [0.4389, 0.8493, 0.5399, 0.6399, 0.1641, 0.3442, 0.4867, 0.7866,
          0.472

Here `sents_emb[i, j]` means the word embedding for the j-th word in the i-th sentence.

In [8]:
#  word embedding for 0-th sentence, 3-rd word
sents_emb[0, 3]

tensor([0.3644, 0.9829, 0.5922, 0.3716, 0.2549, 0.9234, 0.9027, 0.3655, 0.9783,
        0.2015])

### Rank-4 Tensor

As for a rank-4 (four-dimensional) tensor, this could be a batch of sentences, where each word of each sentence is further devided into different heads of attention keys for multi-head attention (you will encounter this in CW2).

In [9]:
sents_emb_keys = torch.rand([2, 5, 4, 10])  # [batch_size = 2, sentence_length = 5, number_of_heads = 4, hidden_size = 10]
                                            # Again, we use random vectors for the purpose of demonstration

Here `sents_emb_keys[i, j, k]` means the attention key vector in the i-th sentence, j-th word, k-th head.

In [10]:
# key vector for 0-th, 3-rd word, 2-nd head
sents_emb_keys[0, 3, 2] 

tensor([0.9995, 0.9725, 0.6199, 0.8235, 0.5397, 0.8774, 0.5437, 0.5569, 0.8486,
        0.3910])

### Caveat: always keep in mind the meaning of shapes of tensors

It is important to always have a record of the meaning of the shape, otherwise one would quickly forget what each tensor dimension means.

In [11]:
sents_emb_keys.size()

torch.Size([2, 5, 4, 10])

Question: what does [2, 5, 4, 10] mean?

Answer: it means [batch_size = 2, sentence_length = 5, number_of_heads = 4, hidden_size = 10], where `number_of_heads` means the number of heads used in the attention mechanism.

## Basic Tensor Operation

### Indexing

In [12]:
# first dimension
sents_emb_keys[0]   # 0-th sentence, all words, all heads, all hidden

tensor([[[0.7381, 0.1543, 0.6069, 0.4136, 0.7161, 0.8884, 0.0661, 0.1811,
          0.1927, 0.2207],
         [0.8948, 0.9163, 0.5317, 0.7555, 0.9061, 0.0091, 0.5229, 0.6696,
          0.7057, 0.6332],
         [0.0599, 0.8027, 0.7089, 0.6296, 0.5882, 0.0374, 0.7048, 0.9837,
          0.5155, 0.5479],
         [0.5107, 0.9159, 0.8990, 0.0545, 0.3603, 0.7753, 0.5407, 0.4403,
          0.8934, 0.4369]],

        [[0.6999, 0.2257, 0.6439, 0.9463, 0.1383, 0.4570, 0.1285, 0.0400,
          0.3239, 0.8844],
         [0.9355, 0.7758, 0.1064, 0.9801, 0.5908, 0.9711, 0.2980, 0.6997,
          0.5886, 0.0251],
         [0.1514, 0.3595, 0.0484, 0.6166, 0.8912, 0.8841, 0.5238, 0.2317,
          0.5468, 0.5227],
         [0.9602, 0.7139, 0.0681, 0.5804, 0.2145, 0.6712, 0.6383, 0.3261,
          0.6275, 0.2552]],

        [[0.3362, 0.3430, 0.9842, 0.8791, 0.8355, 0.3035, 0.6726, 0.7072,
          0.6952, 0.0333],
         [0.8267, 0.4733, 0.2588, 0.2936, 0.6886, 0.3186, 0.3520, 0.0206,
          0.2

In [13]:
sents_emb_keys[0].size()

torch.Size([5, 4, 10])

Q: what does [5, 4, 10] mean?

A: for the 0-th sentence, it has [length = 5, number_of_heads = 4, hidden_size = 10]

In other words, fixing the index of the first dimension of a four-dimensional tensor will result in a three-dimensional tensor.

In [14]:
sents_emb_keys[:, 1, :, :]  # All sentence, 1-st words, all heads, all hidden

tensor([[[0.6999, 0.2257, 0.6439, 0.9463, 0.1383, 0.4570, 0.1285, 0.0400,
          0.3239, 0.8844],
         [0.9355, 0.7758, 0.1064, 0.9801, 0.5908, 0.9711, 0.2980, 0.6997,
          0.5886, 0.0251],
         [0.1514, 0.3595, 0.0484, 0.6166, 0.8912, 0.8841, 0.5238, 0.2317,
          0.5468, 0.5227],
         [0.9602, 0.7139, 0.0681, 0.5804, 0.2145, 0.6712, 0.6383, 0.3261,
          0.6275, 0.2552]],

        [[0.7801, 0.8288, 0.2041, 0.8388, 0.7155, 0.5555, 0.3020, 0.4064,
          0.7264, 0.0538],
         [0.9875, 0.4593, 0.1984, 0.1080, 0.2001, 0.0832, 0.0680, 0.5071,
          0.3675, 0.7010],
         [0.9118, 0.8272, 0.3213, 0.5137, 0.9326, 0.3049, 0.7120, 0.4211,
          0.8940, 0.2103],
         [0.3878, 0.4063, 0.2352, 0.9932, 0.0455, 0.0292, 0.8495, 0.7485,
          0.5724, 0.9278]]])

In [15]:
sents_emb_keys[:, 1, :, :].size()

torch.Size([2, 4, 10])

Q: what does [2, 4, 10] mean?

A: when fixing the 1-st word, we have [batch_size = 2, number_of_heads = 4, hidden_size = 10].

Similarlly we have:

In [16]:
sents_emb_keys[0, :, :, :] # 0-st sentence, all words, all heads, all hidden
sents_emb_keys[:, 0, :, :] # all sentence, 0-th word, all heads, all hidden
sents_emb_keys[:, :, 0, :] # all sentence, all words, 0-th head, all hidden
sents_emb_keys[:, :, :, 0] # all sentence, all words, all heads, 0-th hidden

tensor([[[0.7381, 0.8948, 0.0599, 0.5107],
         [0.6999, 0.9355, 0.1514, 0.9602],
         [0.3362, 0.8267, 0.5742, 0.7995],
         [0.7844, 0.6398, 0.9995, 0.8946],
         [0.9888, 0.9351, 0.8143, 0.7897]],

        [[0.7071, 0.5535, 0.2982, 0.9426],
         [0.7801, 0.9875, 0.9118, 0.3878],
         [0.0035, 0.1969, 0.9288, 0.7128],
         [0.2615, 0.5034, 0.4565, 0.5902],
         [0.6065, 0.7931, 0.7649, 0.7232]]])

In [17]:
# Q: interpret what the following shape means: 
# A: --- YOUR ANSWER HERE ----
print(sents_emb_keys[0, :, :, :].size())
print(sents_emb_keys[:, 0, :, :].size())
print(sents_emb_keys[:, :, 0, :].size())
print(sents_emb_keys[:, :, :, 0].size())

torch.Size([5, 4, 10])
torch.Size([2, 4, 10])
torch.Size([2, 5, 10])
torch.Size([2, 5, 4])


The index can be further fixed at multiple dimensions, for example:

In [18]:
print(sents_emb_keys[0, 1, :, :]) # 0-th sentence, 1-st word, all heads, all hidden
print(sents_emb_keys[0, 1, 2, :]) # 0-th sentence, 1-st word, 2-nd head, all hidden
print(sents_emb_keys[:, 1, 2, :]) # all sentences, 1-st word, 2-nd head, all hidden

tensor([[0.6999, 0.2257, 0.6439, 0.9463, 0.1383, 0.4570, 0.1285, 0.0400, 0.3239,
         0.8844],
        [0.9355, 0.7758, 0.1064, 0.9801, 0.5908, 0.9711, 0.2980, 0.6997, 0.5886,
         0.0251],
        [0.1514, 0.3595, 0.0484, 0.6166, 0.8912, 0.8841, 0.5238, 0.2317, 0.5468,
         0.5227],
        [0.9602, 0.7139, 0.0681, 0.5804, 0.2145, 0.6712, 0.6383, 0.3261, 0.6275,
         0.2552]])
tensor([0.1514, 0.3595, 0.0484, 0.6166, 0.8912, 0.8841, 0.5238, 0.2317, 0.5468,
        0.5227])
tensor([[0.1514, 0.3595, 0.0484, 0.6166, 0.8912, 0.8841, 0.5238, 0.2317, 0.5468,
         0.5227],
        [0.9118, 0.8272, 0.3213, 0.5137, 0.9326, 0.3049, 0.7120, 0.4211, 0.8940,
         0.2103]])


Looking at the shape of the resulting tensors:

In [19]:
sents_emb_keys[:, 1, 2, :].size()

torch.Size([2, 10])

[2, 10] means [batch_size = 2, hidden_size = 10] when fixing the 1-st word and the 2-nd head.

In [20]:
# Q: interpret what the following shape means: 
# A: ---- YOUR ANSWER HERE ---
print(sents_emb_keys[0, 1, :, :].size())
print(sents_emb_keys[0, 1, 2, :].size())

torch.Size([4, 10])
torch.Size([10])


### Slicing

Slicing takes a range of an index within any dimension:

In [21]:
sents_emb_keys[:, 1:4, :, :] # all sentences, 1-3 words (right boundary 4 is not inclusive), all heads, all hidden
sents_emb_keys[:, :4, :, :]  # all sentences, 0-3 words (when left boundary is unspecified, the default is 0), all heads, all hidden
sents_emb_keys[:, 1:, :, :]  # all sentences, 1-last words (when right boundary is unspecified, the default is the last token), all heads, all hidden
sents_emb_keys[:, :, 2:4:, :] # all sentences, all words, 2-3 heads, all hidden
sents_emb_keys[:, :, :, 1:9] # all sentences, all words, all heads, 1-8 hidden

tensor([[[[0.1543, 0.6069, 0.4136, 0.7161, 0.8884, 0.0661, 0.1811, 0.1927],
          [0.9163, 0.5317, 0.7555, 0.9061, 0.0091, 0.5229, 0.6696, 0.7057],
          [0.8027, 0.7089, 0.6296, 0.5882, 0.0374, 0.7048, 0.9837, 0.5155],
          [0.9159, 0.8990, 0.0545, 0.3603, 0.7753, 0.5407, 0.4403, 0.8934]],

         [[0.2257, 0.6439, 0.9463, 0.1383, 0.4570, 0.1285, 0.0400, 0.3239],
          [0.7758, 0.1064, 0.9801, 0.5908, 0.9711, 0.2980, 0.6997, 0.5886],
          [0.3595, 0.0484, 0.6166, 0.8912, 0.8841, 0.5238, 0.2317, 0.5468],
          [0.7139, 0.0681, 0.5804, 0.2145, 0.6712, 0.6383, 0.3261, 0.6275]],

         [[0.3430, 0.9842, 0.8791, 0.8355, 0.3035, 0.6726, 0.7072, 0.6952],
          [0.4733, 0.2588, 0.2936, 0.6886, 0.3186, 0.3520, 0.0206, 0.2313],
          [0.4536, 0.3682, 0.3018, 0.2567, 0.6178, 0.4781, 0.6415, 0.4560],
          [0.9670, 0.3820, 0.9566, 0.9264, 0.7942, 0.2774, 0.8331, 0.3129]],

         [[0.3833, 0.7440, 0.3996, 0.5801, 0.0230, 0.9881, 0.3074, 0.1997],
      

You can also set a stepsize for slicing:

In [22]:
sents_emb_keys[:, 0::2, :, :] # all sentence, 0, 2, 4 words

tensor([[[[0.7381, 0.1543, 0.6069, 0.4136, 0.7161, 0.8884, 0.0661, 0.1811,
           0.1927, 0.2207],
          [0.8948, 0.9163, 0.5317, 0.7555, 0.9061, 0.0091, 0.5229, 0.6696,
           0.7057, 0.6332],
          [0.0599, 0.8027, 0.7089, 0.6296, 0.5882, 0.0374, 0.7048, 0.9837,
           0.5155, 0.5479],
          [0.5107, 0.9159, 0.8990, 0.0545, 0.3603, 0.7753, 0.5407, 0.4403,
           0.8934, 0.4369]],

         [[0.3362, 0.3430, 0.9842, 0.8791, 0.8355, 0.3035, 0.6726, 0.7072,
           0.6952, 0.0333],
          [0.8267, 0.4733, 0.2588, 0.2936, 0.6886, 0.3186, 0.3520, 0.0206,
           0.2313, 0.8176],
          [0.5742, 0.4536, 0.3682, 0.3018, 0.2567, 0.6178, 0.4781, 0.6415,
           0.4560, 0.4921],
          [0.7995, 0.9670, 0.3820, 0.9566, 0.9264, 0.7942, 0.2774, 0.8331,
           0.3129, 0.4751]],

         [[0.9888, 0.7746, 0.3292, 0.4007, 0.0535, 0.5512, 0.4711, 0.5627,
           0.7769, 0.9872],
          [0.9351, 0.0889, 0.8964, 0.0641, 0.2497, 0.4872, 0.4931, 0.

In [23]:
sents_emb_keys[:, 1::2, :, :] # # all sentence, 1, 3 words

tensor([[[[0.6999, 0.2257, 0.6439, 0.9463, 0.1383, 0.4570, 0.1285, 0.0400,
           0.3239, 0.8844],
          [0.9355, 0.7758, 0.1064, 0.9801, 0.5908, 0.9711, 0.2980, 0.6997,
           0.5886, 0.0251],
          [0.1514, 0.3595, 0.0484, 0.6166, 0.8912, 0.8841, 0.5238, 0.2317,
           0.5468, 0.5227],
          [0.9602, 0.7139, 0.0681, 0.5804, 0.2145, 0.6712, 0.6383, 0.3261,
           0.6275, 0.2552]],

         [[0.7844, 0.3833, 0.7440, 0.3996, 0.5801, 0.0230, 0.9881, 0.3074,
           0.1997, 0.7648],
          [0.6398, 0.1023, 0.4721, 0.8809, 0.6355, 0.3216, 0.6054, 0.4593,
           0.6713, 0.1862],
          [0.9995, 0.9725, 0.6199, 0.8235, 0.5397, 0.8774, 0.5437, 0.5569,
           0.8486, 0.3910],
          [0.8946, 0.3388, 0.3985, 0.1377, 0.7580, 0.6391, 0.8913, 0.5509,
           0.4062, 0.5037]]],


        [[[0.7801, 0.8288, 0.2041, 0.8388, 0.7155, 0.5555, 0.3020, 0.4064,
           0.7264, 0.0538],
          [0.9875, 0.4593, 0.1984, 0.1080, 0.2001, 0.0832, 0.0680, 

In [24]:
sents_emb_keys[:, 0:4:2, :, :] # # all sentence, 0, 2 words

tensor([[[[0.7381, 0.1543, 0.6069, 0.4136, 0.7161, 0.8884, 0.0661, 0.1811,
           0.1927, 0.2207],
          [0.8948, 0.9163, 0.5317, 0.7555, 0.9061, 0.0091, 0.5229, 0.6696,
           0.7057, 0.6332],
          [0.0599, 0.8027, 0.7089, 0.6296, 0.5882, 0.0374, 0.7048, 0.9837,
           0.5155, 0.5479],
          [0.5107, 0.9159, 0.8990, 0.0545, 0.3603, 0.7753, 0.5407, 0.4403,
           0.8934, 0.4369]],

         [[0.3362, 0.3430, 0.9842, 0.8791, 0.8355, 0.3035, 0.6726, 0.7072,
           0.6952, 0.0333],
          [0.8267, 0.4733, 0.2588, 0.2936, 0.6886, 0.3186, 0.3520, 0.0206,
           0.2313, 0.8176],
          [0.5742, 0.4536, 0.3682, 0.3018, 0.2567, 0.6178, 0.4781, 0.6415,
           0.4560, 0.4921],
          [0.7995, 0.9670, 0.3820, 0.9566, 0.9264, 0.7942, 0.2774, 0.8331,
           0.3129, 0.4751]]],


        [[[0.7071, 0.4615, 0.3670, 0.2568, 0.9303, 0.3972, 0.1473, 0.4430,
           0.0674, 0.9078],
          [0.5535, 0.0699, 0.0696, 0.1358, 0.7560, 0.5190, 0.4759, 

As you can see, the slicing syntax is `some_tensor[start_index : end_index : step_size]`.

In [25]:
# Q: interpret what the following shape means:
# A: ---- YOUR ANSWER HERE ----
print(sents_emb_keys[:, 1:4, :, :].size())
print(sents_emb_keys[:, :, 3:, :].size())

torch.Size([2, 3, 4, 10])
torch.Size([2, 5, 1, 10])


### Concatenation

To concatenate two tensors within a specified dimension:

In [26]:
sent_1 = torch.tensor([[1, 2, 3]]) # size = [1, 3]
sent_2 = torch.tensor([[4, 5, 6]]) # size = [1, 3]

In [27]:
# concatenate two sentences into a batch
sents = torch.cat([sent_1, sent_2], dim=0)
print(sents)
print(sents.size())

tensor([[1, 2, 3],
        [4, 5, 6]])
torch.Size([2, 3])


In [28]:
# concat the second sentence to the end of the first sentence
sents = torch.cat([sent_1, sent_2], dim=0)
print(sents)
print(sents.size())

tensor([[1, 2, 3],
        [4, 5, 6]])
torch.Size([2, 3])


### Reshaping

Usually reshaping has two effects: (a) spliting tensors or (b) concatenating tensors.

In [29]:
# concatenating tensors
print(sents)
print(sents.size())
sents_cat = sents.view(6)
print(sents_cat)
print(sents_cat.size()) # pay attention how the shape of the tensor changes from [2, 3] -> [6]

tensor([[1, 2, 3],
        [4, 5, 6]])
torch.Size([2, 3])
tensor([1, 2, 3, 4, 5, 6])
torch.Size([6])


In [30]:
# spliting tensors
sents_split_1 = sents_cat.view(2, 3)
print(sents_split_1)
print(sents_split_1.size())
sents_split_2 = sents_cat.view(3, 2)
print(sents_split_2)
print(sents_split_2.size())

tensor([[1, 2, 3],
        [4, 5, 6]])
torch.Size([2, 3])
tensor([[1, 2],
        [3, 4],
        [5, 6]])
torch.Size([3, 2])


### Transposing

Transpose the two dimensions of a tensor:

In [31]:
print(sents) # shape = [batch, length]
sents_transposed = sents.transpose(0, 1)
print(sents_transposed) # shape = [length, batch]

tensor([[1, 2, 3],
        [4, 5, 6]])
tensor([[1, 4],
        [2, 5],
        [3, 6]])


### Caveat: pay close attention to the difference between transposing and reshaping

Generally, reshaping does not change the order of the items within the tensor, while tranposing changes the order.

In [32]:
# view does not change order
print(sents.view(6))
print(sents.view(2, 3))
print(sents.view(3, 2))

# transposing changes the order
print(sents.transpose(0, 1))
print(sents.transpose(0, 1).reshape(6))

tensor([1, 2, 3, 4, 5, 6])
tensor([[1, 2, 3],
        [4, 5, 6]])
tensor([[1, 2],
        [3, 4],
        [5, 6]])
tensor([[1, 4],
        [2, 5],
        [3, 6]])
tensor([1, 4, 2, 5, 3, 6])


## Basic Tensor Computations

### Tensor add scalar

In [33]:
# addition, differences, multiplication, division
sent_emb = torch.rand(3, 5)    # [length = 5, hidden = 10] 
                                # we assume we work with one sentence embedding where there are 3 words and 
                                # each word is associated with an embedding vector with length 5
print(sent_emb)

# the following operations are applied to every entry within the tensor
print(sent_emb + 1)
print(sent_emb - 1)
print(sent_emb * 2)
print(sent_emb / 2)

tensor([[0.7740, 0.6638, 0.9824, 0.7694, 0.7898],
        [0.4934, 0.2439, 0.6094, 0.8491, 0.4725],
        [0.4486, 0.6937, 0.2660, 0.7629, 0.1483]])
tensor([[1.7740, 1.6638, 1.9824, 1.7694, 1.7898],
        [1.4934, 1.2439, 1.6094, 1.8491, 1.4725],
        [1.4486, 1.6937, 1.2660, 1.7629, 1.1483]])
tensor([[-0.2260, -0.3362, -0.0176, -0.2306, -0.2102],
        [-0.5066, -0.7561, -0.3906, -0.1509, -0.5275],
        [-0.5514, -0.3063, -0.7340, -0.2371, -0.8517]])
tensor([[1.5480, 1.3276, 1.9648, 1.5388, 1.5797],
        [0.9867, 0.4878, 1.2188, 1.6981, 0.9451],
        [0.8972, 1.3874, 0.5320, 1.5258, 0.2965]])
tensor([[0.3870, 0.3319, 0.4912, 0.3847, 0.3949],
        [0.2467, 0.1219, 0.3047, 0.4245, 0.2363],
        [0.2243, 0.3469, 0.1330, 0.3815, 0.0741]])


### Vector dot product

In [34]:
# vector dot-product
word_1 = torch.rand(5)
word_2 = torch.rand(5)

prod = (word_1 * word_2).sum() # element-wise multiplication then sum 

### Matrix multiplication

In [35]:
# matrix mulplication
M1 = torch.rand(2, 5)
M2 = torch.rand(5, 10)

prod = torch.matmul(M1, M2)
print(prod.size()) # shape changes: [2, 5] * [5, 10] -> [2, 10]

torch.Size([2, 10])


## Alignment and broadcasting

### Rule 1: if the same shape, then element-wise computation

In [36]:
# adding two words, each word is a vector
word_1 = torch.rand(5) # hidden_size = 5
word_2 = torch.rand(5)
result = word_1 + word_2 # result[i] = word_1[i] + word_2[i] 

In [37]:
# adding two sentences, each sentence is a sequence of word vectors
sent_1 = torch.rand(3, 5) # sent_length = 3, hidden_size = 5
sent_2 = torch.rand(3, 5)
result = sent_1 + sent_2 # result[i, j] = sent_1[i, j] + sent_2[i, j]

### Rule 2: if different shape, first align, then element-wise computation

In [38]:
# Example 1. adding a single word to every word within a sentence
word = torch.tensor([1, 2, 3])              # [hidden = 3]
sent = torch.tensor([[0, 0, 0], 
                     [1, 1, 1]]) # [sent_length = 2, hidden = 3]

result = word.view(1, 3) + sent # underlying process: 
                                # step 1, alignment
                                # word.size() = [1, 3]
                                #                |  |   alignment
                                # sent.size() = [2, 3]
                                #
                                # step 2, repeat where dimension size is 1
                                # word_repeat = word.repeat([2, 1]) -- 0th dim repeat twice, first dim repeat 1 time (= no repeat)
                                #
                                # step 3, align again
                                # align again
                                # word_repeat.size() = [2, 3]
                                #                       |  |   alignmeng, note that the 0th dim of word is repeated
                                # sent.size()        = [2, 3]
                                #
                                # step 4, element-wise addition. 
                                # result = word_repeat + sent
print(result)  # this is basically broadcasing the given word to each word in the given sentence

tensor([[1, 2, 3],
        [2, 3, 4]])


In [39]:
# Example 2. adding two words to two sentences, respectively
words = torch.tensor([[1, 2, 3], [4, 5, 6]])                      # batch_size = 2, hidden_size = 3
sents = torch.tensor([
                        [[0, 0, 0], [1, 1, 1], [2, 2, 2]], 
                        [[3, 3, 3], [4, 4, 4], [5, 5, 5]],        # batch_size = 2, length = 2, hidden_size = 3
                    ])

# we aim to add words[0] to sents[0] and words[1] to sents[1]
result = words.view(2, 1, 3) + sents    # underlying process: 
                                        # 
                                        # step 1, alignment
                                        # words.size() = [2, 1, 3]   two words, each has hidden dim = 3
                                        #                 |  |  |    alignment
                                        # sents.size() = [2, 2, 3]   two sentences, each has two words, each word has hidden dim = 3
                                        # 
                                        # step 2, repeat where dimension size is 1 for broadcasting
                                        # repeat words along 1st dimension
                                        # words_repeat = words.repeat([1, 2, 1]) -- 0th dim repeat 1 time, 1st dim repeat 2 times, 3rd dim repeat 1 time
                                        # 
                                        # step 3, align again
                                        # words_repeat.size() = [2, 2, 3]
                                        #                        |  |  |   alignment, note that the 1st dim of words is repeated
                                        # sents.size()        = [2, 2, 3]
                                        #
                                        # step 4, element-wise addition
                                        # result = words_repeat + sents
print(result)
print(result.size())

tensor([[[ 1,  2,  3],
         [ 2,  3,  4],
         [ 3,  4,  5]],

        [[ 7,  8,  9],
         [ 8,  9, 10],
         [ 9, 10, 11]]])
torch.Size([2, 3, 3])


In [40]:
# Example 3, outer product
# the outer product of two vectors is a matrix
v1 = torch.tensor([1, 2, 3])
v2 = torch.tensor([4, 5, 6])

outer_prod = v1.view(3, 1) * v2.view(1, 3)  # underlying process: 
                                            # 
                                            # step 1, alignment
                                            # v1.size() = [3, 1]  
                                            #              |  |  alignment
                                            # v2.size() = [1, 3] 
                                            # 
                                            # step 2, repeat where dimension size is 1
                                            # repeat v1 along 1st dimension
                                            # v1_repeat = v1.repeat([1, 3]) -- 0th dim repeat 1 time, 1st repeat 3 times
                                            # repeat v2 along 0th dimension
                                            # v2_repeat = v2.repeat([3, 1]) -- 0th dim repeat 3 times, 1st repeat 1 time
                                            #
                                            # step 3, align again and element-wise computation
                                            # v1_repeat.size() = [3, 3]
                                            #                     |  | 
                                            # v2_repeat.size() = [3, 3]
                                            #
                                            # step 4, elementwise multiplication
                                            # outer_prod = v1_repeat * v2_repeat

print(outer_prod)

tensor([[ 4,  5,  6],
        [ 8, 10, 12],
        [12, 15, 18]])


In [41]:
# Pay attention to the shape of the tensor, again
v1 = torch.tensor([1, 2, 3])
v2 = torch.tensor([4, 5, 6])

# Q: what is the differences between the following two operations: 
# A: ---- YOUR ANSWER HERE ----
result_1 = v1.view(3, 1) * v2.view(1, 3)
result_2 = v1.view(1, 3) * v2.view(3, 1)

# Section 2. Batchified Tensor Computation

In this section, we will study how to compute with tensors whose first dimension is the batch size.

## Batch matrix multiplication

In [42]:
# linear transform of the word embeddings of two sentences
sents = torch.rand(2, 5, 10) # [batch_size = 2, length = 2, hidden_size= 10]
weight = torch.rand(10, 20)  # we transform each word from a length-10 vector to a length-20 vector

sents_transform = torch.matmul(sents, weight)   # underlying process:
                                                # sents.size() = [2,  5, 10]
                                                #                 |       |         alignment. 
                                                # view weight as [1,     10, 20]    repeat where dimension is 1
                                                # matrix multiplication happens at the final two dimensions, [2, 5, 10] x [10, 20] -> [2, 5, 20]
print(sents_transform.size())

torch.Size([2, 5, 20])


## Retrieving embedding vectors from an embedding matrix

In [43]:
# in deep learning practice, usually we start with sequences of word index
sent = torch.tensor([[10, 25, 59, 77, 88],  # suppose [10, 25, 59, 77, 88] means = ['Oh', I', 'really', 'like', 'cats']
                     [16, 29, 40, 56, 3]]   # suppose [16, 29, 40, 56, 3] means = ['Jack', 'does', 'not', 'have', 'dogs']
                    )
print(sent.size())

# neural networks requires words being represented as embedding vectors, not index. So we store an embedding matrix for every word
vocab_size = 100
hidden_size = 10
embedding_matrix = torch.rand([vocab_size, hidden_size])

# converting index to embeddings consists of two steps:
# 1. convert index representation to one-hot representation
sent_one_hot = F.one_hot(sent, vocab_size).float()
# 2. batch matrix multiplication between the one hot representation and the embedding matrix
#    NOTE: when multiply a one-hot vector to a matrix, the one-hot vector essentially retrieves the corresponding row vector from the matrix
sent_emb = torch.matmul(sent_one_hot, embedding_matrix) # underlying process:
                                                        # sent_one_hot.size() =    [2,  5, 100]
                                                        #                           |       |         alignment. 
                                                        # view embedding_matrix as [1,     100, 10]   repeat where dimension is 1
                                                        # matrix multiplication happens at the final two dimensions, [2, 5, 100] x [100, 10] -> [2, 5, 10]
                                                        # In this case, multiplying an one-hot vector to a matrix means to retrieve the corresponding 
                                                        # row vector from the matrix

print(sent_emb.size())

# the following show the embedding vector from word index 10 is retrieved for sent[0, 0]
print(sent_emb[0, 0])
print(embedding_matrix[10])

# similarlly the embedding vector from word index 25 is retrieved for sent[0, 1]
print(sent_emb[0, 1])
print(embedding_matrix[25])

# ---- YOUR TASK ----
# verify how one-hot vector can retrieve the corresponding row vector from the matrix

torch.Size([2, 5])
torch.Size([2, 5, 10])
tensor([0.4093, 0.6592, 0.9869, 0.9115, 0.9096, 0.0590, 0.6847, 0.3976, 0.3881,
        0.0565])
tensor([0.4093, 0.6592, 0.9869, 0.9115, 0.9096, 0.0590, 0.6847, 0.3976, 0.3881,
        0.0565])
tensor([0.3722, 0.2414, 0.6395, 0.0657, 0.0763, 0.2267, 0.9993, 0.7255, 0.1229,
        0.5108])
tensor([0.3722, 0.2414, 0.6395, 0.0657, 0.0763, 0.2267, 0.9993, 0.7255, 0.1229,
        0.5108])


## Linear transform of the word embeddings

In [44]:
# linear transform of the word embeddings of two sentences where each sentence has 4 attention heads
# this computation is performed in multi-head attention in CW2

sents = torch.rand(2, 5, 4, 10) # [batch_size = 2, length = 2, number_of_heads = 4, hidden_size= 10]
weight = torch.rand(10, 20)  # we transform each word from a length-10 vector to a length-20 vector

sents_transform = torch.matmul(sents, weight)   # underlying process:
                                                # sents.size() = [2,  5, 4, 10]
                                                #                 |   |      |         alignment. 
                                                # view weight as [1,  1,    10, 20]    repeat where dimension is 1
                                                # matrix multiplication happens at the final two dimensions, [2, 5, 4, 10] * [10, 20] -> [2, 5, 4, 20]
print(sents_transform.size())

torch.Size([2, 5, 4, 20])


## Similarity: one single vector v.s. a batch of sentence 

In [45]:
# similarity: one single vector v.s. a batch of sentence representations
query = torch.rand(10)
sents = torch.rand(2, 5, 10)

# suppose we would like to compute the dot-product (as a measure of similarity) between the query vector and all words within all sentences
similarity = (query.view(1, 1, 10) * sents).sum(dim=2)
print(similarity.size())  # similarity[i, j] means the vector dot-product between the query vector and the word j at sentence i

torch.Size([2, 5])


## Similarity: a batch of vectors v.s. a batch of sentence  

In [46]:
# similarity: a batch of vectors v.s. a batch of sentence representations 
query = torch.rand(2, 10) # pay attention to the differences to the previous case
sents = torch.rand(2, 5, 10)

similarity = (query.view(2, 1, 10) * sents).sum(dim=2)
print(similarity.size())  # similarity[i, j] means the vector dot-product between the query[i] and the word j at sentence i

torch.Size([2, 5])


## Similarity: a sentence v.s. a batch of sentence

In [47]:
# similarity: a sentence v.s. a batch of sentence representations
sent_0 = torch.rand(4, 10)
sents = torch.rand(2, 5, 10)

# Q: what is the underlying process? hint: recall the previous align-and-broadcast
# A: ---- YOUR ANSWER HERE ----
similarity = (sent_0.view(1, 4, 1, 10) * sents.view(2, 1, 5, 10)).sum(dim=3) 
print(similarity.size()) # similarity[i, j, k] means the vector dot-product between sent_0[j] (j-th word) and sents[i, k] (i-th sent, k-th word)

similarity = (sent_0.view(1, 1, 4, 10) * sents.view(2, 5, 1, 10)).sum(dim=3)
print(similarity.size()) # similarity[i, j, k] means the vector dot-product between sent_0[k] (k-th word) and sents[i, j] (i-th sent, j-th word)

# NOTE: pay attention two the differences of the shapes

torch.Size([2, 4, 5])
torch.Size([2, 5, 4])


## Similarity: two batches of sentences

In [48]:
# similarity: two batches of sentences representations
# this is usually happens in machine translation where we have a batch of sentences of source language, and another batch of sentences of target language
source_sents = torch.rand(2, 5, 10) # [batch_size = 2, source_sent_length = 5, hidden_size = 10]
target_sents = torch.rand(2, 6, 10) # [batch_size = 2, target_sent_length = 6, hidden_size = 10]

# Q: what is the underlying process? hint: recall the previous align-and-broadcast
# A: ---- YOUR ANSWER HERE ----
similarity = (source_sents.view(2, 5, 1, 10) * target_sents.view(2, 1, 6, 10)).sum(dim = 3)
print(similarity.size()) # similarity[i, j, k] means source_sents[i, j] v.s. target_sents[i, k]

similarity = (source_sents.view(2, 1, 5, 10) * target_sents.view(2, 6, 1, 10)).sum(dim = 3)
print(similarity.size()) # similarity[i, j, k] means source_sents[i, k] v.s. target_sents[i, j]
# NOTE: pay attention two the differences of the shapes

torch.Size([2, 5, 6])
torch.Size([2, 6, 5])


# Section 3. Masked Batch Computation

When putting sentences of difference length into a batch, common practice is to pad them into the same pre-specified maximum length with a special PAD token, then mask out computation involving the PAD token.

## Basic Masking

In [49]:
# a batch of sentences with different length
PAD = 0
sents = torch.tensor([[1, 2, 3, 0, 0], # length = 3, 2 PAD tokens,
                      [3, 4, 0, 0, 0], # length = 2, 3 PAD tokens
                      [5, 6, 7, 8, 9]] # length = 5, no PAD token
                    )

In [50]:
# create a batch of mask vectors
mask = (sents != PAD).float()
print(mask)

tensor([[1., 1., 1., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1.]])


## Masked Batchified Computation

Now we repeat the computation of the previous section. But this time, we mask out the computation involving the PAD tokens.

## Similarity: one single vector vs. a batch of sentence + mask

In [51]:
# one single vector v.s. a batch of sentence representations + a batch of mask
query = torch.rand(10)
sents = torch.rand(3, 5, 10)

# suppose we would like to compute the dot-product (as a measure of similarity) between the query vector and all words within all sentences
similarity = (query.view(1, 1, 10) * sents * mask.view(3, 5, 1)).sum(dim=2) # query.size = [1, 1, 10]
                                                                            # sents.size = [3, 5, 10]
                                                                            # mask.size  = [3, 5, 1]
                                                                            # recall the align-then-broadcast introduced in the previous section
print(similarity)
print(similarity.size())  # similarity[i, j] means the vector dot-product between the query vector and the word j at sentence i
                          # similarity scores for PAD tokens are masked to be 0

tensor([[2.4386, 2.3735, 2.5634, 0.0000, 0.0000],
        [2.1971, 2.1193, 0.0000, 0.0000, 0.0000],
        [2.1219, 1.9663, 2.0173, 2.1244, 2.0675]])
torch.Size([3, 5])


## Similarity: a sentence vs. a batch of sentence + mask

In [52]:
# similarity: a sentence v.s. a batch of sentence representations
sent_0 = torch.rand(4, 10)
sent_0_mask = torch.tensor([1, 1, 1, 0]) # we assume the length of sent_0 is 3

sents = torch.rand(3, 5, 10)

# Q: what is the underlying process? hint: recall the previous align-and-broadcast
# A: ---- YOUR ANSWER HERE ----
#
# hint: first mask out sent_0
#       sent_0.size        = [1, 4, 1, 10]
#       sent_0_mask.size   = [1, 4, 1, 1]
#       sent_0_masked.size = [1, 4, 1, 10]
#
#       then mask out sents
#       sents.size         = [3, 1, 5, 10]
#       mask.size          = [3, 1, 5, 1 ]
#       sents_masked.size  = [3, 1, 5, 10]
#
#       finally computed similarity score for the two masked tensors
#       sent_0_masked.size = [1, 4, 1, 10]
#       sents_masked.size  = [3, 1, 5, 10]
#       similarity.size    = [3, 4, 5]

similarity = (sent_0.view(1, 4, 1, 10) * sent_0_mask.view(1, 4, 1, 1) * sents.view(3, 1, 5, 10) * mask.view(3, 1, 5, 1)).sum(dim=3) 

print(similarity)
print(similarity.size()) # similarity[i, j, k] means the vector dot-product between sent_0[j] (j-th word) and sents[i, k] (i-th sent, k-th word)
                         # similarity scores for PAD tokens are masked to be 0

tensor([[[2.3704, 2.3730, 2.7710, 0.0000, 0.0000],
         [2.1014, 2.5274, 2.1251, 0.0000, 0.0000],
         [1.7304, 2.2811, 2.0582, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

        [[4.3135, 3.4149, 0.0000, 0.0000, 0.0000],
         [3.9689, 3.0051, 0.0000, 0.0000, 0.0000],
         [2.5112, 2.6538, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

        [[3.4162, 3.4895, 2.9795, 2.2999, 3.3681],
         [3.1975, 3.3605, 2.6317, 1.8804, 3.3257],
         [2.7647, 2.7293, 1.7944, 2.0281, 2.5565],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]]])
torch.Size([3, 4, 5])


## Similarity: two batches of masked sentences

In [53]:
# similarity: two batches of sentences representations
# this is usually happens in machine translation where we have a batch of sentences of source language, and another batch of sentences of target language
# here additionaly, we assume each sentence has its own mask 
source_sents = torch.rand(2, 5, 10) # [batch_size = 2, source_sent_length = 5, hidden_size = 10]
source_mask  = torch.tensor([[1, 1, 1, 1, 0], # length = 4
                             [1, 1, 1, 1, 1]] # length = 5
                           )
target_sents = torch.rand(2, 6, 10) # [batch_size = 2, target_sent_length = 6, hidden_size = 10]
target_mask  = torch.tensor([[1, 1, 1, 0, 0, 0], # length = 3
                             [1, 1, 1, 1, 1, 1]] # length = 6
                           )

# Q: what is the underlying process? hint: recall the previous align-and-broadcast-and-mask
# A: ---- YOUR ANSWER HERE ----
similarity = (source_sents.view(2, 5, 1, 10) * source_mask.view(2, 5, 1, 1) * target_sents.view(2, 1, 6, 10) * target_mask.view(2, 1, 6, 1)).sum(dim = 3)
print(similarity)
print(similarity.size()) # similarity[i, j, k] means source_sents[i, j] v.s. target_sents[i, k]

tensor([[[3.9366, 2.8278, 2.2431, 0.0000, 0.0000, 0.0000],
         [3.2524, 2.4841, 2.1404, 0.0000, 0.0000, 0.0000],
         [3.0136, 2.5326, 1.6738, 0.0000, 0.0000, 0.0000],
         [2.8650, 2.0855, 2.2605, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

        [[2.4893, 2.9497, 2.8121, 2.2709, 3.2474, 3.6408],
         [2.1631, 2.0666, 1.6209, 1.8893, 1.6808, 2.8438],
         [1.5273, 2.1812, 1.6632, 2.0251, 1.5746, 2.9743],
         [2.6392, 2.5629, 2.1693, 2.3413, 1.9995, 3.4827],
         [1.5420, 1.0035, 1.1882, 1.3498, 1.6223, 1.6428]]])
torch.Size([2, 5, 6])


# Section 4. Final Application: Negative Log Likelihood

Finally, we compute the per-word negative log likelihood of a neural language model that puts everything that we have learned together.

The per-word NLL for a sentence $x_1, ..., x_T$ is:

$$
- \frac{1}{T} \sum_t \log p(x_t | x_{1:t-1})
$$

which is the average of word likelihood over sentence length.

In [54]:
# we start with the final layer of a neural language model which output a tensor named logits (logits means the vector before putting into softmax):
logits = torch.rand(2, 5, 100)  # [batch_size = 2, max_sent_len = 5, vocab_size = 100]
                                # in practice this should be the output of the final layer of a neural language model. Here we just simulate it as random tensor
vocab_size = 100 # in practice the vocabulary size is usually >10K, here for demonstration we simplify it to 100

# putting the logits into a softmax function gives the probability of each word within the vocabulary
probs = F.softmax(logits, dim=2)

# negative log likelihood requires the log of probability
log_probs = F.log_softmax(logits, dim=2)

# a language model aims to maximize the probability of each word in every sentence, but the PAD token should be excluded
target_sent = torch.tensor([[10, 25, 59, 77, 0],   # length = 4, suppose [10, 25, 59, 77, 0] means = ['I', 'really', 'like', 'cats', 'PAD']
                            [16, 29, 40, 56, 3]]   # length = 5, suppose [16, 29, 40, 56, 3] means = ['Jack', 'does', 'not', 'have', 'dogs']
                          )

# transform the index representation to the one-hot representation
target_one_hot = F.one_hot(target_sent, vocab_size)

# create mask for the target sentence
mask = (target_sent != 0).float()

# the following computation computes the negative log likelihood
# ---- YOUR TASK ----
# 1. annotate the shape of every tensor in the following computation
# 2. explain every computation step
negative_log_likelihood = (((-target_one_hot * log_probs).sum(dim=2) * mask).sum(dim=1) / mask.sum(dim=1)).mean()

# Further Reading

Up to this stage, you have already suffered from endless annotation of shape of tensor. Is there a way that one can write code which is self-explanatory, rather than requires shape annotation at every single line? 

The answer is yes, and the solution is [einsum](https://rockt.github.io/2018/04/30/einsum) and [einops](https://einops.rocks/). You are encouraged to find them out yourself.